In [1]:
# Cell 1: Mount Drive & install YOLOv5
from google.colab import drive
drive.mount('/content/drive')

# clone YOLOv5 and install requirements
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Mounted at /content/drive
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17485, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 17485 (delta 80), reused 31 (delta 31), pack-reused 17376 (from 3)
Receiving objects: 100% (17485/17485), 16.33 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (11984/11984), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%%bash
cat <<EOT > /content/data.yaml
train: /content/drive/MyDrive/AI_Project_Dataset/Training/images
val:   /content/drive/MyDrive/AI_Project_Dataset/Validation/images
test:  /content/drive/MyDrive/AI_Project_Dataset/Testing/images

nc: 4
names: ['Corrosion_Steel','Corrosion_Bolt','Crack','Missing_Bolt']
EOT


In [3]:
# Cell 3: Define dataset paths & hyperparameters
import os

ROOT        = "/content/drive/MyDrive/AI_Project_Dataset"
TRAIN_IMG   = os.path.join(ROOT, "Training/images")
TRAIN_LBL   = os.path.join(ROOT, "Training/labels")
VAL_IMG     = os.path.join(ROOT, "Validation/images")
VAL_LBL     = os.path.join(ROOT, "Validation/labels")
TEST_IMG    = os.path.join(ROOT, "Testing/images")
TEST_LBL    = os.path.join(ROOT, "Testing/labels")

PROJECT             = os.path.join(ROOT, "YOLOv5_Runs")
FINAL_MODEL_DIR     = os.path.join(ROOT, "Final_Model")
ANNOTATED_TEST_DIR  = os.path.join(ROOT, "Test_Annotated")

os.makedirs(PROJECT, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)
os.makedirs(ANNOTATED_TEST_DIR, exist_ok=True)

EPOCHS     = 50
BATCH_SIZE = 16


In [4]:
# Cell 4: Utility functions — box conversion, IoU, GT loader
import numpy as np
from pathlib import Path

def xywhn2xyxy(x, w, h):
    xc, yc, wn, hn = x
    return [
        (xc - wn/2) * w,
        (yc - hn/2) * h,
        (xc + wn/2) * w,
        (yc + hn/2) * h
    ]

def compute_iou(box1, box2):
    xa, ya = max(box1[0], box2[0]), max(box1[1], box2[1])
    xb, yb = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter  = max(0, xb - xa) * max(0, yb - ya)
    area1  = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2  = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union  = area1 + area2 - inter
    return inter/union if union > 0 else 0.0

def load_gt_boxes(label_path, img_w, img_h):
    gt = []
    for line in Path(label_path).read_text().splitlines():
        vals = line.split()
        if len(vals) != 5:
            continue
        # drop vals[0] = class_id
        x_c, y_c, w_n, h_n = map(float, vals[1:])
        gt.append(xywhn2xyxy((x_c, y_c, w_n, h_n), img_w, img_h))
    return gt


In [5]:
# Cell 5: Train, validate, detect, compute IoU, loop with max 5 iterations and 50% IoU threshold
import os
import csv
from pathlib import Path
from PIL import Image

os.environ['WANDB_MODE'] = 'offline'
prev_weights   = "yolov5s.pt"
MAX_ITERS      = 5
IOU_THRESHOLD  = 0.50
TARGET_RATIO   = 0.90
iteration      = 0

while True:
    iteration += 1
    run_name     = f"iter_{iteration}"
    weights_path = os.path.join(PROJECT, run_name, "weights", "best.pt")

    # 1) Train (checkpoint every epoch)
    get_ipython().system(f'''
    python train.py \
      --img 640 \
      --batch {BATCH_SIZE} \
      --epochs {EPOCHS} \
      --data /content/data.yaml \
      --weights {prev_weights} \
      --project {PROJECT} \
      --name {run_name} \
      --save-period 1 \
      --exist-ok
    ''')

    # 2) Validate on validation set
    get_ipython().system(f'python val.py --data /content/data.yaml --weights {weights_path}')

    # 3) Inference on test set (save labels and confidences)
    test_run = f"test_{iteration}"
    get_ipython().system(f'''
    python detect.py \
      --weights {weights_path} \
      --source {TEST_IMG} \
      --project {PROJECT} \
      --name {test_run} \
      --exist-ok \
      --save-txt \
      --save-conf
    ''')

    # 4) Compute IoU and collect results
    detect_dir = Path(PROJECT) / test_run / "labels"
    output_csv = Path(PROJECT) / test_run / "iou_results.csv"
    rows = []

    for img_path in Path(TEST_IMG).glob("*.*"):
        label_file = Path(TEST_LBL) / f"{img_path.stem}.txt"
        if not label_file.exists():
            continue

        w, h      = Image.open(img_path).size
        gt_boxes  = load_gt_boxes(label_file, w, h)
        pred_file = detect_dir / f"{img_path.stem}.txt"

        if not pred_file.exists():
            rows.append([img_path.name, 0.0, 0.0])
        else:
            for line in pred_file.read_text().splitlines():
                cls, x, y, wn, hn, conf = map(float, line.split())
                pred_box = xywhn2xyxy((x, y, wn, hn), w, h)
                best_iou = max((compute_iou(pred_box, g) for g in gt_boxes), default=0.0)
                rows.append([img_path.name, conf, best_iou])

    # Save IoU results to CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["image_name", "confidence", "IoU"])
        writer.writerows(rows)

    # 5) Stopping criterion
    ious   = [r[2] for r in rows]
    total  = len(ious)
    passed = sum(iou >= IOU_THRESHOLD for iou in ious)
    ratio  = passed / total if total > 0 else 0

    print(f"Iteration {iteration}: {passed}/{total} entries with IoU ≥ {IOU_THRESHOLD} ({ratio:.1%})")

    if ratio >= TARGET_RATIO:
        print("Stopping: reached target ratio of test images.")
        break

    if iteration >= MAX_ITERS:
        print("Stopping: reached maximum iterations.")
        break

    prev_weights = weights_path


Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      20/49      4.35G    0.05291    0.07256   0.001536        298        640:  74% 20/27 [00:35<00:16,  2.43s/it]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      20/49      4.35G    0.05289    0.07205     0.0015        261        640:  78% 21/27 [00:38<00:16,  2.72s/it]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      20/49      4.35G    0.05311    0.07199   0.001485        313        640:  81% 22/27 [00:39<00:10,  2.14s/it]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):


In [7]:
# Cell 6: Save the final model and annotated test images back to Drive
import os
import shutil

# Last iteration index from Cell 5
last_iter = iteration

# 1) Copy the final best.pt
best_model_src = os.path.join(PROJECT, f"iter_{last_iter}", "weights", "best.pt")
best_model_dst = os.path.join(FINAL_MODEL_DIR, "yolov5_defect_detector.pt")
shutil.copy(best_model_src, best_model_dst)

# 2) Copy the annotated test images
# detect.py saved them directly under PROJECT/test_<n>/
annotated_src = os.path.join(PROJECT, f"test_{last_iter}")
annotated_dst = ANNOTATED_TEST_DIR
shutil.copytree(annotated_src, annotated_dst, dirs_exist_ok=True)

print(f"Final model saved to: {best_model_dst}")
print(f"Annotated test images copied to: {annotated_dst}")


Final model saved to: /content/drive/MyDrive/AI_Project_Dataset/Final_Model/yolov5_defect_detector.pt
Annotated test images copied to: /content/drive/MyDrive/AI_Project_Dataset/Test_Annotated
